In [1]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("avaliev/chat_doctor")

README.md:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/120M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/95588 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11949 [00:00<?, ? examples/s]

In [3]:
print(dataset)
print(dataset["train"][0])  # View the first example

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 95588
    })
    validation: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 11949
    })
    test: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 11949
    })
})
{'input': 'Hi.  My son had an appendectomy within the passed 24 hours.  Previous to that he had been sick with stomach virus type symptoms for 5 days.  He now has had two loose bowel movements within the past four hours that has blood in them.  Is this to be expected or do I need to take him back to the ER?  Ive called the Dr.s answering service and am awaiting a response.', 'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'output': 'Hello dear, Thank you for your contact to Chat Doctor. I read and understand your concern. I am Chat Doctor answering your concern. Yes, you need to visit to the

In [6]:
from transformers import T5Tokenizer

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocess function
def preprocess_function(examples):
    # Format the input with a task-specific prefix
    inputs = [f"As a doctor, respond to the following patient's medical question: {input_text}" for input_text in examples["input"]]
    targets = examples["output"]

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/95588 [00:00<?, ? examples/s]

Map:   0%|          | 0/11949 [00:00<?, ? examples/s]

Map:   0%|          | 0/11949 [00:00<?, ? examples/s]

In [7]:
from transformers import T5ForConditionalGeneration

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=500,
    push_to_hub=False,  # Set to True if you want to upload the model to Hugging Face Hub
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

<ipython-input-9-94497ec1db77>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ericrlessa (ericrlessa-george-brown-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.025800,0.952712


TrainOutput(global_step=11949, training_loss=1.0766949849564098, metrics={'train_runtime': 8707.3704, 'train_samples_per_second': 10.978, 'train_steps_per_second': 1.372, 'total_flos': 1.2937052119105536e+16, 'train_loss': 1.0766949849564098, 'epoch': 1.0})

In [10]:
trainer.save_model("./fine_tuned_t5")

In [12]:
tokenizer.save_pretrained("./fine_tuned_t5")

('./fine_tuned_t5/tokenizer_config.json',
 './fine_tuned_t5/special_tokens_map.json',
 './fine_tuned_t5/spiece.model',
 './fine_tuned_t5/added_tokens.json')

In [13]:
!!pip install transformers datasets huggingface_hub

['Requirement already satisfied: transformers in /usr/local/lib/python3.11/dist-packages (4.47.1)',
 'Requirement already satisfied: datasets in /usr/local/lib/python3.11/dist-packages (3.2.0)',
 'Requirement already satisfied: huggingface_hub in /usr/local/lib/python3.11/dist-packages (0.27.1)',
 'Requirement already satisfied: filelock in /usr/local/lib/python3.11/dist-packages (from transformers) (3.17.0)',
 'Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.11/dist-packages (from transformers) (1.26.4)',
 'Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.11/dist-packages (from transformers) (24.2)',
 'Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.11/dist-packages (from transformers) (6.0.2)',
 'Requirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.11/dist-packages (from transformers) (2024.11.6)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.11/dist-packages (from transform

In [37]:
from huggingface_hub import login

# Log in to Hugging Face Hub
login()

In [ ]:
!huggingface-cli whoami


In [34]:
!huggingface-cli logout
!huggingface-cli login

Successfully logged out from all access tokens.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: fineGrained).
The token `all` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store)

In [29]:
!ls fine_tuned_t5

added_tokens.json  generation_config.json  special_tokens_map.json  tokenizer_config.json
config.json	   model.safetensors	   spiece.model		    training_args.bin


In [40]:
!git clone https://huggingface.co/Eric-Lessa/t5_chatbot_doctor


Cloning into 't5_chatbot_doctor'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (4/4), 1.13 KiB | 1.13 MiB/s, done.


In [44]:
!cp -r /content/fine_tuned_t5/* .


In [49]:
!git config --global user.email "xyz@gmail.com"
!git config --global user.name "Xyz"



In [50]:
!git add .
!git commit -m "Pushing fine-tuned model"
!git push


[main 5c71f5d] Pushing fine-tuned model
 8 files changed, 1245 insertions(+)
 create mode 100644 added_tokens.json
 create mode 100644 config.json
 create mode 100644 generation_config.json
 create mode 100644 model.safetensors
 create mode 100644 special_tokens_map.json
 create mode 100644 spiece.model
 create mode 100644 tokenizer_config.json
 create mode 100644 training_args.bin
Uploading LFS objects: 100% (3/3), 243 MB | 64 MB/s, done.
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (10/10), done.
Writing objects: 100% (10/10), 3.51 KiB | 3.51 MiB/s, done.
Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/Eric-Lessa/t5_chatbot_doctor
   d6c6de2..5c71f5d  main -> main


In [ ]:
from transformers import pipeline

generator = pipeline("summarization", model="Eric-Lessa/t5_chatbot_doctor")

def generate_response(question):
  result = generator("generate response" + question, max_length=500)
  print(result)

generate_response("What should I do if my child experiences stomach symptoms and has blood in their stool after an appendectomy?")

In [17]:
from transformers import pipeline

generator = pipeline("text2text-generation", model="Eric-Lessa/t5_chatbot_doctor")

def generate_response(question):
  result = generator("generate response" + question, max_length=500)
  print(result)

generate_response("What should I do if my child experiences stomach symptoms and has blood in their stool after an appendectomy?")

Device set to use cpu
